## Install and import relevant packages

In [1]:
! pip install dropbox
! pip install dropboxdrivefs
! pip install pyhdf

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy
import pooch
import dropbox
import dropboxdrivefs as dbx
import logging
import tarfile
import gzip
import shutil
from pyhdf.SD import SD, SDC

## Load GEOTRACES GP16 TEI data

In [3]:
fs = dbx.DropboxDriveFileSystem(token='u19t2wI3eH4AAAAAAAAAAdplJXSWJIRd1Rp8sAl1MIcxTxz4fkJ8xP_y8dqS7sdv')
file = fs.open('/RN-20211117121141_CF2D9BA9D51D3B7CE0538486ABC0F5FD/idp2021/GEOTRACES_IDP2021_v1_seawater_netcdf.zip')
file


INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


<File-like object DropboxDriveFileSystem, https://ucc13eb5f581ea321faf4a0ef0c8.dl.dropboxusercontent.com/cd/0/get/BbUjSRrQ4E9RazaRBCyIVtIdg-J3H09iqdrHeyp_hjgwgXStxbjGjAKtdsPoAPJV9zCUeOxNCXdqouxkBDEILS16QgCpws7C5sdHKLmPPK0Egss6c089M_7E_PhsKAHef2NNdc0oNSbDpWgF0UnRjL0P/file>

## Load SIMPLE-TRIM model output

In [4]:
simple_trim_url = 'http://tdevries.eri.ucsb.edu/wp-content/uploads/2018/03/SIMPLE_TRIM_output.nc'

file_path = pooch.retrieve(url=simple_trim_url,
                          known_hash='md5:75487e25bfe7448736883a26b697ba81')
file_path

'/home/jovyan/.cache/pooch/e4a0a4525472949384e03b3f13eddc14-SIMPLE_TRIM_output.nc'

In [5]:
trim = xr.open_dataset(file_path)
trim = trim.set_coords(['LAT', 'LON'])
trim

<xarray.Dataset>
Dimensions:      (depth: 24, longitude: 180, latitude: 91, version: 12)
Coordinates:
    LAT          (depth, longitude, latitude) float64 ...
    LON          (depth, longitude, latitude) float64 ...
Dimensions without coordinates: depth, longitude, latitude, version
Data variables: (12/51)
    mask         (depth, longitude, latitude) float64 ...
    DEPTH        (depth, longitude, latitude) float64 ...
    Volume       (depth, longitude, latitude) float64 ...
    Area         (longitude, latitude) float64 ...
    NPP          (version, longitude, latitude) float64 ...
    Zeu          (depth, longitude, latitude) float64 ...
    ...           ...
    k_SL         (version) float64 ...
    k_SR         (version) float64 ...
    k_R          (version) float64 ...
    npp_version  (version) float64 ...
    phistar      (version) float64 ...
    mind_large   (version) float64 ...

## Load VGPM SST and NPP Data

In [6]:
VGPM2010_url = 'http://orca.science.oregonstate.edu/data/1x2/8day/vgpm.r2018.m.chl.m.sst/hdf/vgpm.m.2010.tar'
VGPM2011_url = 'http://orca.science.oregonstate.edu/data/1x2/8day/vgpm.r2018.m.chl.m.sst/hdf/vgpm.m.2011.tar'

file_path2010 = pooch.retrieve(url=VGPM2010_url,
                              known_hash='md5:f58c846cfeb3de5cad7a28210283ad67')
file_path2011 = pooch.retrieve(url=VGPM2011_url,
                              known_hash='md5:ec20fd84a5ef81ba4cdaf2cd80e336d0')

In [7]:
tarfile_2010 = tarfile.open(file_path2010, 'r:')
npp2010_files = tarfile_2010.getnames()
#npp2010_filesunzipped = []
for file in npp2010_files:
    with gzip.open(file, 'rb') as f_in, open(file[:-3], 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        #npp2010_filesunzipped.append(f_out)
        #f = SD(f_out, SDC.READ)
# these are now ready to use when TEI data is loaded